In [23]:
import numpy as np

# mne imports
import mne
from mne import io, read_epochs, concatenate_epochs
from mne.datasets import sample
from mne import Epochs, create_info, events_from_annotations
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.time_frequency import AverageTFR

# EEGNet-specific imports
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# tools for plotting confusion matrices
from matplotlib import pyplot as plt

In [2]:
cd epochs

/Users/jang-wonjun/Documents/eeg/ml/epochs


## grip 

In [5]:
cd grip 

/Users/jang-wonjun/Documents/eeg/ml/epochs/grip


In [6]:
ls

epoch0-epo.fif  epoch2-epo.fif  epoch4-epo.fif
epoch1-epo.fif  epoch3-epo.fif  epoch5-epo.fif


In [7]:
n_epochs = 6
epoch_list = [f'epoch{i}-epo.fif' for i in range(n_epochs)]

In [9]:
epoch_grip = concatenate_epochs([read_epochs(epoch_i) for epoch_i in epoch_list])
epoch_grip

Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/grip/epoch0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/grip/epoch1-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/grip/epoch2-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/grip/epoch3-epo.fif ...
    Found the data of int

Number of events,60
Events,finger_movement: 60
Time range,-2.000 – 2.000 s
Baseline,off


## pinch

In [10]:
cd ../pinch

/Users/jang-wonjun/Documents/eeg/ml/epochs/pinch


In [11]:
ls

epoch0-epo.fif  epoch1-epo.fif  epoch2-epo.fif  epoch3-epo.fif  epoch4-epo.fif


In [12]:
n_epochs = 5
epoch_list = [f'epoch{i}-epo.fif' for i in range(n_epochs)]

In [13]:
epoch_pinch = concatenate_epochs([read_epochs(epoch_i) for epoch_i in epoch_list])
epoch_pinch

Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/pinch/epoch0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/pinch/epoch1-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/pinch/epoch2-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/pinch/epoch3-epo.fif ...
    Found the data of

Number of events,50
Events,finger_movement: 50
Time range,-2.000 – 2.000 s
Baseline,off


In [14]:
fmin, fmax =2 ,8
#fmin,fmax=None,None
epoch_pinch = epoch_pinch.copy().filter(fmin,fmax)
epoch_grip = epoch_grip.copy().filter(fmin,fmax)

Setting up band-pass filter from 2 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 213 samples (1.664 s)

Setting up band-pass filter from 2 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter len

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 350 out of 350 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 420 out of 420 | elapsed:    0.0s finished


In [17]:
# label 0 for pinch and label 1 for grip
w_tmin, w_tmax = -1, 1
X_grip = epoch_grip.crop(w_tmin,w_tmax)
X_pinch = epoch_pinch.crop(w_tmin,w_tmax)
X = np.concatenate([X_grip.get_data(),X_pinch.get_data()],axis=0)*1000
y = np.array([1 for i in range(X_grip.get_data().shape[0])]+[0 for i in range(X_pinch.get_data().shape[0])])

In [18]:
X.shape, y.shape

((110, 7, 257), (110,))

In [19]:
kernels, chans, samples = 1, 7, 257

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

In [21]:
X_val,X_test,y_val,y_test = train_test_split(X_test, y_test,test_size=0.5, random_state=42)

In [24]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
y_val = np_utils.to_categorical(y_val)

In [25]:
model = EEGNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = 0.5, kernLength = 32, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')

model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

In [26]:
numParams    = model.count_params()

In [27]:
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                               save_best_only=True)

In [28]:
class_weights = {0:1, 1:1}

In [29]:
fittedModel = model.fit(X_train, y_train, batch_size = 16, epochs = 300, 
                        verbose = 2, validation_data=(X_val, y_val),
                        callbacks=[checkpointer], class_weight = class_weights)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.69027, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.7072 - accuracy: 0.5114 - val_loss: 0.6903 - val_accuracy: 0.8182 - 420ms/epoch - 70ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.69027 to 0.68836, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6966 - accuracy: 0.4773 - val_loss: 0.6884 - val_accuracy: 0.8182 - 56ms/epoch - 9ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.68836 to 0.68726, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6801 - accuracy: 0.5909 - val_loss: 0.6873 - val_accuracy: 0.8182 - 56ms/epoch - 9ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.68726 to 0.68596, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6675 - accuracy: 0.6250 - val_loss: 0.6860 - val_accuracy: 0.8182 - 57ms/epoch - 10ms/step
Epoch 5/300


/opt/homebrew/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 5: val_loss improved from 0.68596 to 0.68476, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6797 - accuracy: 0.5682 - val_loss: 0.6848 - val_accuracy: 0.8182 - 57ms/epoch - 10ms/step
Epoch 6/300

Epoch 6: val_loss did not improve from 0.68476
6/6 - 0s - loss: 0.6677 - accuracy: 0.5682 - val_loss: 0.6853 - val_accuracy: 0.8182 - 42ms/epoch - 7ms/step
Epoch 7/300

Epoch 7: val_loss did not improve from 0.68476
6/6 - 0s - loss: 0.6606 - accuracy: 0.6023 - val_loss: 0.6849 - val_accuracy: 0.8182 - 42ms/epoch - 7ms/step
Epoch 8/300

Epoch 8: val_loss improved from 0.68476 to 0.68470, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.6647 - accuracy: 0.5909 - val_loss: 0.6847 - val_accuracy: 0.8182 - 58ms/epoch - 10ms/step
Epoch 9/300

Epoch 9: val_loss did not improve from 0.68470
6/6 - 0s - loss: 0.6607 - accuracy: 0.6023 - val_loss: 0.6851 - val_accuracy: 0.8182 - 44ms/epoch - 7ms/step
Epoch 10/300

Epoch 10: val_loss did not improve from 0.68470
6/6 - 0s - loss: 0.6352

Epoch 53/300

Epoch 53: val_loss did not improve from 0.68470
6/6 - 0s - loss: 0.2903 - accuracy: 0.8636 - val_loss: 0.7651 - val_accuracy: 0.1818 - 44ms/epoch - 7ms/step
Epoch 54/300

Epoch 54: val_loss did not improve from 0.68470
6/6 - 0s - loss: 0.2566 - accuracy: 0.9205 - val_loss: 0.7567 - val_accuracy: 0.1818 - 42ms/epoch - 7ms/step
Epoch 55/300

Epoch 55: val_loss did not improve from 0.68470
6/6 - 0s - loss: 0.2591 - accuracy: 0.9318 - val_loss: 0.7400 - val_accuracy: 0.1818 - 42ms/epoch - 7ms/step
Epoch 56/300

Epoch 56: val_loss did not improve from 0.68470
6/6 - 0s - loss: 0.3127 - accuracy: 0.8636 - val_loss: 0.7830 - val_accuracy: 0.1818 - 43ms/epoch - 7ms/step
Epoch 57/300

Epoch 57: val_loss did not improve from 0.68470
6/6 - 0s - loss: 0.2948 - accuracy: 0.8977 - val_loss: 0.8054 - val_accuracy: 0.1818 - 45ms/epoch - 7ms/step
Epoch 58/300

Epoch 58: val_loss did not improve from 0.68470
6/6 - 0s - loss: 0.2425 - accuracy: 0.8977 - val_loss: 0.7873 - val_accuracy: 0.181

Epoch 96/300

Epoch 96: val_loss improved from 0.32505 to 0.31530, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.3154 - accuracy: 0.8523 - val_loss: 0.3153 - val_accuracy: 0.9091 - 57ms/epoch - 9ms/step
Epoch 97/300

Epoch 97: val_loss did not improve from 0.31530
6/6 - 0s - loss: 0.2152 - accuracy: 0.9091 - val_loss: 0.3385 - val_accuracy: 1.0000 - 42ms/epoch - 7ms/step
Epoch 98/300

Epoch 98: val_loss did not improve from 0.31530
6/6 - 0s - loss: 0.2051 - accuracy: 0.9318 - val_loss: 0.3453 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 99/300

Epoch 99: val_loss did not improve from 0.31530
6/6 - 0s - loss: 0.1340 - accuracy: 0.9886 - val_loss: 0.3256 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 100/300

Epoch 100: val_loss did not improve from 0.31530
6/6 - 0s - loss: 0.1862 - accuracy: 0.9318 - val_loss: 0.3182 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 101/300

Epoch 101: val_loss improved from 0.31530 to 0.29141, saving model to /tmp/checkpoint.h5

Epoch 137/300

Epoch 137: val_loss improved from 0.08818 to 0.08367, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.1740 - accuracy: 0.9432 - val_loss: 0.0837 - val_accuracy: 1.0000 - 56ms/epoch - 9ms/step
Epoch 138/300

Epoch 138: val_loss improved from 0.08367 to 0.08085, saving model to /tmp/checkpoint.h5
6/6 - 0s - loss: 0.1291 - accuracy: 0.9545 - val_loss: 0.0808 - val_accuracy: 1.0000 - 54ms/epoch - 9ms/step
Epoch 139/300

Epoch 139: val_loss did not improve from 0.08085
6/6 - 0s - loss: 0.1798 - accuracy: 0.9432 - val_loss: 0.0811 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 140/300

Epoch 140: val_loss did not improve from 0.08085
6/6 - 0s - loss: 0.1252 - accuracy: 0.9659 - val_loss: 0.0825 - val_accuracy: 1.0000 - 43ms/epoch - 7ms/step
Epoch 141/300

Epoch 141: val_loss did not improve from 0.08085
6/6 - 0s - loss: 0.1277 - accuracy: 0.9659 - val_loss: 0.0832 - val_accuracy: 1.0000 - 42ms/epoch - 7ms/step
Epoch 142/300

Epoch 142: val_loss improved from 0.0808

Epoch 182/300

Epoch 182: val_loss did not improve from 0.05642
6/6 - 0s - loss: 0.1709 - accuracy: 0.9205 - val_loss: 0.0571 - val_accuracy: 1.0000 - 47ms/epoch - 8ms/step
Epoch 183/300

Epoch 183: val_loss did not improve from 0.05642
6/6 - 0s - loss: 0.1806 - accuracy: 0.9318 - val_loss: 0.0592 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 184/300

Epoch 184: val_loss did not improve from 0.05642
6/6 - 0s - loss: 0.2574 - accuracy: 0.8750 - val_loss: 0.0622 - val_accuracy: 1.0000 - 42ms/epoch - 7ms/step
Epoch 185/300

Epoch 185: val_loss did not improve from 0.05642
6/6 - 0s - loss: 0.0854 - accuracy: 1.0000 - val_loss: 0.0574 - val_accuracy: 1.0000 - 42ms/epoch - 7ms/step
Epoch 186/300

Epoch 186: val_loss did not improve from 0.05642
6/6 - 0s - loss: 0.1364 - accuracy: 0.9773 - val_loss: 0.0577 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 187/300

Epoch 187: val_loss did not improve from 0.05642
6/6 - 0s - loss: 0.1667 - accuracy: 0.9659 - val_loss: 0.0602 - val_acc

6/6 - 0s - loss: 0.0928 - accuracy: 0.9773 - val_loss: 0.0760 - val_accuracy: 1.0000 - 43ms/epoch - 7ms/step
Epoch 230/300

Epoch 230: val_loss did not improve from 0.05642
6/6 - 0s - loss: 0.1339 - accuracy: 0.9773 - val_loss: 0.0675 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 231/300

Epoch 231: val_loss did not improve from 0.05642
6/6 - 0s - loss: 0.2163 - accuracy: 0.9318 - val_loss: 0.0639 - val_accuracy: 1.0000 - 40ms/epoch - 7ms/step
Epoch 232/300

Epoch 232: val_loss did not improve from 0.05642
6/6 - 0s - loss: 0.0766 - accuracy: 0.9886 - val_loss: 0.0611 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 233/300

Epoch 233: val_loss did not improve from 0.05642
6/6 - 0s - loss: 0.1501 - accuracy: 0.9318 - val_loss: 0.0613 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 234/300

Epoch 234: val_loss did not improve from 0.05642
6/6 - 0s - loss: 0.0925 - accuracy: 0.9659 - val_loss: 0.0631 - val_accuracy: 1.0000 - 40ms/epoch - 7ms/step
Epoch 235/300

Epoch 235: 

6/6 - 0s - loss: 0.1187 - accuracy: 0.9659 - val_loss: 0.0510 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 276/300

Epoch 276: val_loss did not improve from 0.04298
6/6 - 0s - loss: 0.0789 - accuracy: 0.9886 - val_loss: 0.0512 - val_accuracy: 1.0000 - 43ms/epoch - 7ms/step
Epoch 277/300

Epoch 277: val_loss did not improve from 0.04298
6/6 - 0s - loss: 0.2002 - accuracy: 0.9205 - val_loss: 0.0514 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 278/300

Epoch 278: val_loss did not improve from 0.04298
6/6 - 0s - loss: 0.1258 - accuracy: 0.9659 - val_loss: 0.0564 - val_accuracy: 1.0000 - 40ms/epoch - 7ms/step
Epoch 279/300

Epoch 279: val_loss did not improve from 0.04298
6/6 - 0s - loss: 0.1072 - accuracy: 0.9773 - val_loss: 0.0639 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 280/300

Epoch 280: val_loss did not improve from 0.04298
6/6 - 0s - loss: 0.1053 - accuracy: 0.9659 - val_loss: 0.0682 - val_accuracy: 1.0000 - 41ms/epoch - 7ms/step
Epoch 281/300

Epoch 281: 

In [30]:
# load optimal weights
model.load_weights('/tmp/checkpoint.h5')

In [31]:
probs       = model.predict(X_test)
preds       = probs.argmax(axis = -1)  
acc         = np.mean(preds == y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))

1/1 [==============================] - 0s 97ms/step
Classification accuracy: 1.000000 


In [33]:
preds

array([0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0])

In [35]:
y_test

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)